In [1]:
from pyspark.sql import functions as F 
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.storagelevel import StorageLevel

In [3]:
spark.conf.set("spark.network.timeout","10000000")
spark.conf.set("spark.executor.heartbeatInterval", "200000")

In [4]:
 |-- batters: struct (nullable = true)
 |    |-- batter: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- type: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- ppu: double (nullable = true)
 |-- topping: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- type: string (nullable = true)

In [5]:
df_json.select("id","type","name","ppu","topping","batters.*")\
       .withColumn("zipped", F.explode(F.arrays_zip("batter","topping")))\
       .select("id","type","name","ppu","zipped.*").show()

In [6]:
list= [[3,'2020-03-31',5],
 [2,'2020-03-24',6],
 [1,'2020-03-17',3],
 [1,'2020-03-12',2],
 [1,'2020-03-08',1]]

df=spark.createDataFrame(list, ['id','date','some_count'])

df.show()

+---+----------+----------+
 id| date|some_count|
+---+----------+----------+
 3|2020-03-31| 5|
 2|2020-03-24| 6|
 1|2020-03-17| 3|
 1|2020-03-12| 2|
 1|2020-03-08| 1|
+---+----------+----------+

In [7]:
w=Window.orderBy(F.col("date"))
df.withColumn("count_change", F.lag("some_count").over(w))\
  .withColumn("count_change", F.when(F.col("count_change").isNotNull(), F.col("some_count")-F.col("count_change"))\
              .otherwise(F.lit(0))).orderBy(F.col("date").desc()).show()

+---+----------+----------+------------+
 id| date|some_count|count_change|
+---+----------+----------+------------+
 3|2020-03-31| 5| -1|
 2|2020-03-24| 6| 3|
 1|2020-03-17| 3| 1|
 1|2020-03-12| 2| 1|
 1|2020-03-08| 1| 0|
+---+----------+----------+------------+

In [8]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w=Window.orderBy(F.col("date").asc())

df.withColumn("count_change", F.lag("some_count").over(w))\
  .withColumn("count_change", F.when(F.col("count_change").isNotNull(), F.col("some_count")-F.col("count_change"))\
              .otherwise(F.lit(0))).orderBy(F.col("date").desc()).show()

+---+----------+----------+------------+
 id| date|some_count|count_change|
+---+----------+----------+------------+
 3|2020-03-31| 5| -1|
 2|2020-03-24| 6| 3|
 1|2020-03-17| 3| 1|
 1|2020-03-12| 2| 1|
 1|2020-03-08| 1| 0|
+---+----------+----------+------------+

In [9]:
import pyspark.sql.functions as F
from pyspark.sql import Window

l = [
(  1,  1,          0),
(  2,  2,          0),
(  3,  3,          0),
(  4,  4,          1),
(  5, 10,          0),
(  6, 14,          0),
(  7, 13,          1),
(  8, 18,          1),
(  9, 23,          0),
( 10,  5,          0),
( 11, 56,          0),
( 12, 66,          0),
( 13, 34,          0),
( 14, 35,          0),
( 15, 36,          0),
( 16, 37,          0),
( 17, 39,          1),
( 18, 54,          1),
( 19, 68,          1),
( 20, 90,          0)
]

columns = ['Id','seq','seq_checker']

df=spark.createDataFrame(l, columns)
df = df.withColumn( "check_sequence", F.lit( 0 ) )

w = Window.partitionBy( ).orderBy( "id" ).rangeBetween( 0, 2 )

output = df.withColumn( 'check_sequence', F.when( F.max( F.col( "seq_checker" ) ).over( w ) == 1, F.col( "check_sequence" ) + 1 ) \
                                                  .otherwise( F.col( "check_sequence" ) ) )

output.show()

+---+---+-----------+--------------+
 Id|seq|seq_checker|check_sequence|
+---+---+-----------+--------------+
 1| 1| 0| 0|
 2| 2| 0| 1|
 3| 3| 0| 1|
 4| 4| 1| 1|
 5| 10| 0| 1|
 6| 14| 0| 1|
 7| 13| 1| 1|
 8| 18| 1| 1|
 9| 23| 0| 0|
 10| 5| 0| 0|
 11| 56| 0| 0|
 12| 66| 0| 0|
 13| 34| 0| 0|
 14| 35| 0| 0|
 15| 36| 0| 1|
 16| 37| 0| 1|
 17| 39| 1| 1|
 18| 54| 1| 1|
 19| 68| 1| 1|
 20| 90| 0| 0|
+---+---+-----------+--------------+

In [10]:
list=[['2019','12','28']]

df=spark.createDataFrame(list,['year','month','day'])
df.show()

+----+-----+---+
year|month|day|
+----+-----+---+
2019| 12| 28|
+----+-----+---+

In [11]:
df.withColumn("date", F.to_date(F.concat_ws("-",*(x for x in ['year','month','day'])),"yyyy-MM-dd")).show()

+----+-----+---+----------+
year|month|day| date|
+----+-----+---+----------+
2019| 12| 28|2019-12-28|
+----+-----+---+----------+

In [12]:
list=[['a','01-01-19',45],
     ['a','03-01-19',89],
      ['a','04-01-19',24],
      ['a','05-01-19',778],
      ['b','02-01-19',78],
      ['b','03-01-19',37],
      ['c','01-01-19',49],
      ['c','02-01-19',24],
      ['c','04-01-19',11]]

df=spark.createDataFrame(list, ['tag','ts','value'])

list2=[['01-01-19'],
       ['02-01-19'],
       ['03-01-19'],
       ['04-01-19'],
       ['05-01-19']]
df1=spark.createDataFrame(list2,['ts'])


df.show()
df1.show()

+---+--------+-----+
tag| ts|value|
+---+--------+-----+
 a|01-01-19| 45|
 a|03-01-19| 89|
 a|04-01-19| 24|
 a|05-01-19| 778|
 b|02-01-19| 78|
 b|03-01-19| 37|
 c|01-01-19| 49|
 c|02-01-19| 24|
 c|04-01-19| 11|
+---+--------+-----+

+--------+
 ts|
+--------+
01-01-19|
02-01-19|
03-01-19|
04-01-19|
05-01-19|
+--------+

In [13]:
spark.conf.set("spark.sql.crossJoin.enabled", "true")
df2=df.join(df1.groupBy().agg(F.collect_list("ts").alias("ts1")))
df3=df2.groupBy("tag").agg(F.collect_list("ts").alias("ts"),F.collect_list("value").alias("value"),F.first("ts1").alias("ts1"))
df3.withColumn("ts2", F.array_except("ts1","ts")).drop("ts1")\
   .withColumn("value", F.flatten(F.array(F.expr("""transform(ts2, x-> '')"""),"value")))\
   .withColumn("ts", F.array_union("ts2","ts")).drop("ts2")\
   .withColumn("zip", F.explode(F.arrays_zip("ts","value")))\
   .select("tag","zip.*")\
   .orderBy("tag","ts").show()

+---+--------+-----+
tag| ts|value|
+---+--------+-----+
 a|01-01-19| 45|
 a|02-01-19| |
 a|03-01-19| 89|
 a|04-01-19| 24|
 a|05-01-19| 778|
 b|01-01-19| |
 b|02-01-19| 78|
 b|03-01-19| 37|
 b|04-01-19| |
 b|05-01-19| |
 c|01-01-19| 49|
 c|02-01-19| 24|
 c|03-01-19| |
 c|04-01-19| 11|
 c|05-01-19| |
+---+--------+-----+

In [14]:
spark.conf.set("spark.sql.crossJoin.enabled", "true")
df2=df.groupBy("tag").agg(F.collect_list("ts").alias("ts"),F.collect_list("value").alias("value"))\
      .join(df1.groupBy().agg(F.collect_list("ts").alias("ts1")))
df2.show(truncate=False)
'''df2.withColumn("ts2", F.array_except("ts1","ts")).drop("ts1")\
   .withColumn("value", F.flatten(F.array(F.expr("""transform(ts2, x-> '')"""),"value")))\
   .withColumn("ts", F.array_union("ts2","ts")).drop("ts2")\
   .withColumn("zip", F.explode(F.arrays_zip("ts","value")))\
   .select("tag","zip.*")\
   .orderBy("tag","ts").show()'''

+---+----------------------------------------+-----------------+------------------------------------------------------+
tag|ts |value |ts1 |
+---+----------------------------------------+-----------------+------------------------------------------------------+
c |[01-01-19, 02-01-19, 04-01-19] |[49, 24, 11] |[[[01-01-19, 02-01-19, 03-01-19, 04-01-19, 05-01-19]]]|
b |[02-01-19, 03-01-19] |[78, 37] |[[[01-01-19, 02-01-19, 03-01-19, 04-01-19, 05-01-19]]]|
a |[01-01-19, 03-01-19, 04-01-19, 05-01-19]|[45, 89, 24, 778]|[[[01-01-19, 02-01-19, 03-01-19, 04-01-19, 05-01-19]]]|
+---+----------------------------------------+-----------------+------------------------------------------------------+

Out[14]: 'df2.withColumn("ts2", F.array_except("ts1","ts")).drop("ts1") .withColumn("value", F.flatten(F.array(F.expr("""transform(ts2, x-> \'\')"""),"value"))) .withColumn("ts", F.array_union("ts2","ts")).drop("ts2") .withColumn("zip", F.explode(F.arrays_zip("ts","value"))) .select("tag","zip.*") .orderBy("tag","ts").show()'

In [15]:
df3.withColumn("ts2", F.array_except("ts1","ts")).drop("ts1")\
   .withColumn("value", F.flatten(F.array(F.expr("""transform(ts2, x-> '')"""),"value")))\
   .withColumn("ts", F.array_union("ts2","ts")).drop("ts2")\
   .withColumn("zip", F.explode(F.arrays_zip("ts","value")))\
   .select("tag","zip.*")\
   .orderBy("tag","ts").show()

+---+--------+-----+
tag| ts|value|
+---+--------+-----+
 a|01-01-19| 45|
 a|02-01-19| |
 a|03-01-19| 89|
 a|04-01-19| 24|
 a|05-01-19| 778|
 b|01-01-19| |
 b|02-01-19| 78|
 b|03-01-19| 37|
 b|04-01-19| |
 b|05-01-19| |
 c|01-01-19| 49|
 c|02-01-19| 24|
 c|03-01-19| |
 c|04-01-19| 11|
 c|05-01-19| |
+---+--------+-----+

In [16]:
df1.alias("t1").join(df.alias("t2"),F.col("t1.ts")==F.col("t2.ts"),"left").\
selectExpr("coalesce(t1.ts,t2.ts) as ts","tag","value").\
orderBy("ts").\
show()

+--------+---+-----+
 ts|tag|value|
+--------+---+-----+
01-01-19| a| 45|
01-01-19| c| 49|
02-01-19| b| 78|
02-01-19| c| 24|
03-01-19| a| 89|
03-01-19| b| 37|
04-01-19| a| 24|
04-01-19| c| 11|
05-01-19| a| 778|
+--------+---+-----+

In [18]:
startdate = '2017-01-01'
enddate = '2018-01-01'
pushdown_query="(select * from dbo.Employee where CreatedDate > '"+startdate+"' and CreatedDate < '"+enddate+"') emp_alias"  
print(pushdown_query)

(select * from dbo.Employee where CreatedDate > '2017-01-01' and CreatedDate < '2018-01-01') emp_alias

In [19]:
list=[['2018-02-08 11:04']]

df=spark.createDataFrame(list,['timstm_hm'])
df.show()

+----------------+
 timstm_hm|
+----------------+
2018-02-08 11:04|
+----------------+

In [20]:
from pyspark.sql.functions import col, unix_timestamp
df.withColumn('timstm_hm_timestamp', unix_timestamp(col('timstm_hm'), "yyyy-MM-dd HH:mm").cast("timestamp")).show()

+----------------+-------------------+
 timstm_hm|timstm_hm_timestamp|
+----------------+-------------------+
2018-02-08 11:04|2018-02-08 11:04:00|
+----------------+-------------------+

In [21]:
from pyspark.sql.functions import to_timestamp
df.withColumn("timestm_hm_timestamp", to_timestamp("timstm_hm","yyyy-MM-dd HH:mm" )).show()

+----------------+--------------------+
 timstm_hm|timestm_hm_timestamp|
+----------------+--------------------+
2018-02-08 11:04| 2018-02-08 11:04:00|
+----------------+--------------------+

In [22]:
list=[['2018-02-08 11:04']]

df=spark.createDataFrame(list,['timstm_hm'])
df.show()

+----------------+
 timstm_hm|
+----------------+
2018-02-08 11:04|
+----------------+

In [23]:
list= [['12345-123-12345-4'],
       ['5678-4321-123-12']]

df=spark.createDataFrame(list,['column_to_split'])
df.show()

+-----------------+
 column_to_split|
+-----------------+
12345-123-12345-4|
 5678-4321-123-12|
+-----------------+

In [24]:
from pyspark.sql import functions as sf
df.withColumn("result", sf.element_at(sf.split("column_to_split","\-"),-1).cast("int")).show()

+-----------------+------+
 column_to_split|result|
+-----------------+------+
12345-123-12345-4| 4|
 5678-4321-123-12| 12|
+-----------------+------+

In [25]:
from pyspark.sql import functions as sf
df.withColumn("result", (sf.reverse(sf.split("column_to_split","\-"))[0]).cast("int")).show()

+-----------------+------+
 column_to_split|result|
+-----------------+------+
12345-123-12345-4| 4|
 5678-4321-123-12| 12|
+-----------------+------+

In [27]:
df.withColumn("result", int(sf.col("column_to_split").split('-')[-1])).show()

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-3603488501737188> in <module> 
 ----> 1 df . withColumn ( "result" , int ( sf . col ( "column_to_split" ) . split ( '-' ) [ - 1 ] ) ) . show ( ) 

 TypeError : 'Column' object is not callable

In [28]:
list=[[[9,2500], 27],
      [[7,1900], 13],
      [[1,1800], 9],
      [[8,1600], 6],
      [[6,1500], 5]]

df=spark.createDataFrame(list,['tuple','value'])

df.show()

+---------+-----+
 tuple|value|
+---------+-----+
[9, 2500]| 27|
[7, 1900]| 13|
[1, 1800]| 9|
[8, 1600]| 6|
[6, 1500]| 5|
+---------+-----+

In [29]:
list=[[9,2500, 27],
      [7,1900, 13],
      [1,1800, 9],
      [8,1600, 6],
      [6,1500, 5]]

df=spark.createDataFrame(list,['OccYear','value1','value'])

df.show()

+-------+------+-----+
OccYear|value1|value|
+-------+------+-----+
 9| 2500| 27|
 7| 1900| 13|
 1| 1800| 9|
 8| 1600| 6|
 6| 1500| 5|
+-------+------+-----+

In [30]:
df=df.withColumn("tuple", F.struct("OccYear","value1")).drop("OccYear","value1").select("tuple","value")

In [31]:
value=10
df.withColumn("element", F.when(F.col("value")>=F.lit(value), F.col("tuple.value1")).otherwise(F.lit(None)))\
  .agg(F.mean("element").alias("Avg"),F.lit(value).alias("value")).show()

+------+-----+
 Avg|value|
+------+-----+
2200.0| 10|
+------+-----+

In [32]:
list=[[1, 27],
      [2, 13],
      [3, 9],
      [4, 6],
      [5, 5]]

df=spark.createDataFrame(list,['tuple','value'])

df.show()

+-----+-----+
tuple|value|
+-----+-----+
 1| 27|
 2| 13|
 3| 9|
 4| 6|
 5| 5|
+-----+-----+

In [33]:
df.withColumn("struct", F.struct(F.col("tuple"),F.col("value"))).withColumn("struct1", F.col("struct.value")).show()

+-----+-----+-------+-------+
tuple|value| struct|struct1|
+-----+-----+-------+-------+
 1| 27|[1, 27]| 27|
 2| 13|[2, 13]| 13|
 3| 9| [3, 9]| 9|
 4| 6| [4, 6]| 6|
 5| 5| [5, 5]| 5|
+-----+-----+-------+-------+

In [34]:
value=10
df.withColumn("element", F.when(F.col("value")>=F.lit(value), F.element_at("tuple",2)).otherwise(F.lit(None)))\
  .agg(F.mean("element").alias("Avg"),F.lit(value).alias("value")).show()

+------+-----+
 Avg|value|
+------+-----+
2200.0| 10|
+------+-----+

In [35]:
value=10
df.withColumn("element", F.when(F.col("value")>=F.lit(value), F.col("tuple")[1]).otherwise(F.lit(None)))\
  .agg(F.mean("element").alias("Avg"),F.lit(value).alias("value")).show()

+------+-----+
 Avg|value|
+------+-----+
2200.0| 10|
+------+-----+

In [36]:
df.withColumn("element", F.col("tuple")[1]).show()

+---------+-----+-------+
 tuple|value|element|
+---------+-----+-------+
[9, 2500]| 27| 2500|
[7, 1900]| 13| 1900|
[1, 1800]| 9| 1800|
[8, 1600]| 6| 1600|
[6, 1500]| 5| 1500|
+---------+-----+-------+

In [37]:
dict = {'Id1': 'string1', 'Id2': 'string2', 'Id3': 'string3', 'Id4': 'string2', 'Id5': 'string4', 'Id6': 'string2', 'Id7': 'string3'}
idlist = ['Id2', 'Id3', 'Id5', 'Id6', 'Id7']

final=[]
for i in idlist:
  d=[]
  val=dict[i]
  d.append(i)
  for key,value in dict.items():
    if key!=i and val==value:
      d.append(key)

      
  final.append(d)
        
final



Out[32]: [['Id2', 'Id4', 'Id6'],
 ['Id3', 'Id7'],
 ['Id5'],
 ['Id6', 'Id2', 'Id4'],
 ['Id7', 'Id3']]

In [38]:
people=[('john', 35, 54, 'A'), ('george', 94, 84, 'B'), ('nicolas', 7, 9, 'B'), ('tom', 86, 93, 'A'), ('jason', 62, 73, 'B'), ('bill', 15, 58, 'A'), ('william', 9, 3, 'A'), ('brad', 73, 37, 'B'), ('cosmo', 52, 67, 'B'), ('jerry', 73, 30, 'A')]

df=spark.createDataFrame(people, ['name','x','y','class'])



In [39]:
from pyspark.sql import functions as F
df1=df.repartition(100,F.col("class")).persist() #forexample:100.This should be computed(total number of cores in cluster *2)
df2=df1.filter("class='A'")
df3=df1.filter("class='B'").withColumnRenamed("x","x1").withColumnRenamed("y","y1").withColumnRenamed("name","name1")

df4=df2.join(df3,df2['class']!=df3['class'])
df4.withColumn("distance", F.sqrt(F.pow(F.col("x")-F.col("x1"),F.lit(2))+\
                                 F.pow(F.col("y")-F.col("y1"),F.lit(2))))\
   .filter("distance<10").select("name","name1").explain()

== Physical Plan ==
*(2) Project [name#22529, name1#23394]
+- *(2) BroadcastNestedLoopJoin BuildRight, Inner, ((SQRT((POWER(cast((x#22530L - x1#23384L) as double), 2.0) + POWER(cast((y#22531L - y1#23389L) as double), 2.0))) < 10.0) && NOT (class#22532 = class#23402))
 :- *(2) Filter (isnotnull(class#22532) && (class#22532 = A))
 : +- InMemoryTableScan [name#22529, x#22530L, y#22531L, class#22532], [isnotnull(class#22532), (class#22532 = A)]
 : +- InMemoryRelation [name#22529, x#22530L, y#22531L, class#22532], StorageLevel(disk, memory, 1 replicas)
 : +- Exchange hashpartitioning(class#22532, 100), [id=#2768]
 : +- InMemoryTableScan [name#22529, x#22530L, y#22531L, class#22532]
 : +- InMemoryRelation [name#22529, x#22530L, y#22531L, class#22532], StorageLevel(disk, memory, 1 replicas)
 : +- Exchange hashpartitioning(class#22532, 150), [id=#2665]
 : +- InMemoryTableScan [name#22529, x#22530L, y#22531L, class#22532]
 : +- InMemoryRelation [name#22529, x#22530L, y#22531L, class#22532], StorageLevel(disk, memory, 1 replicas)
 : +- Exchange hashpartitioning(class#22532, 100), [id=#2562]
 : +- InMemoryTableScan [name#22529, x#22530L, y#22531L, class#22532]
 : +- InMemoryRelation [name#22529, x#22530L, y#22531L, class#22532], StorageLevel(disk, memory, 1 replicas)
 : +- Exchange hashpartitioning(class#22532, 200), [id=#2392]
 : +- *(1) Scan ExistingRDD[name#22529,x#22530L,y#22531L,class#22532]
 +- BroadcastExchange IdentityBroadcastMode, [id=#2938]
 +- *(1) Project [name#23399 AS name1#23394, x#23400L AS x1#23384L, y#23401L AS y1#23389L, class#23402]
 +- *(1) Filter (isnotnull(class#23402) && (class#23402 = B))
 +- InMemoryTableScan [class#23402, name#23399, x#23400L, y#23401L], [isnotnull(class#23402), (class#23402 = B)]
 +- InMemoryRelation [name#23399, x#23400L, y#23401L, class#23402], StorageLevel(disk, memory, 1 replicas)
 +- Exchange hashpartitioning(class#22532, 100), [id=#2768]
 +- InMemoryTableScan [name#22529, x#22530L, y#22531L, class#22532]
 +- InMemoryRelation [name#22529, x#22530L, y#22531L, class#22532], StorageLevel(disk, memory, 1 replicas)
 +- Exchange hashpartitioning(class#22532, 150), [id=#2665]
 +- InMemoryTableScan [name#22529, x#22530L, y#22531L, class#22532]
 +- InMemoryRelation [name#22529, x#22530L, y#22531L, class#22532], StorageLevel(disk, memory, 1 replicas)
 +- Exchange hashpartitioning(class#22532, 100), [id=#2562]
 +- InMemoryTableScan [name#22529, x#22530L, y#22531L, class#22532]
 +- InMemoryRelation [name#22529, x#22530L, y#22531L, class#22532], StorageLevel(disk, memory, 1 replicas)
 +- Exchange hashpartitioning(class#22532, 200), [id=#2392]
 +- *(1) Scan ExistingRDD[name#22529,x#22530L,y#22531L,class#22532]

In [40]:
dict['Id2']

Out[9]: 'string2'

In [41]:
for keys,values in dict.items():
   print(keys,values)

Id1 string1
Id2 string2
Id3 string3
Id4 string2
Id5 string4
Id6 string2
Id7 string3

In [42]:
outlist = [['Id2', 'Id6'], ['Id3', 'Id7']]

In [43]:

list=[['123k',['BLACK', 'black']],
     ['123k',['WHITE',None]],
     ['124k',['BLUE',""]]]

df=spark.createDataFrame(list,['key','_value'])

df.show()

+----+--------------+
 key| _value|
+----+--------------+
123k|[BLACK, black]|
123k| [WHITE,]|
124k| [BLUE, ]|
+----+--------------+

In [44]:
df.withColumn("boolean", F.array_contains("_value", "")).show()

+----+---------------+-------+
 key| _value|boolean|
+----+---------------+-------+
123k|[BLACK, black,]| null|
123k| [WHITE]| false|
124k| [BLUE, ,]| true|
+----+---------------+-------+

In [45]:
df.withColumn("boolean", F.array_contains("_value", ""))\
  .filter(~(F.col("boolean")==False)).drop("boolean").show()

+----+--------+
 key| _value|
+----+--------+
124k|[BLUE, ]|
+----+--------+

In [46]:
df.withColumn("boolean", F.expr("""exists(_value, x-> x is null or x=='')"""))\
  .filter((F.col("boolean")==True)).drop("boolean").show()

+----+--------+
 key| _value|
+----+--------+
123k|[WHITE,]|
124k|[BLUE, ]|
+----+--------+

In [47]:
df.withColumn("_value2", F.expr("""filter(_value, x-> x is not null and x!='')"""))\
  .filter((F.size("_value2")==F.size("_value"))).drop("_value2").show()

+----+--------------+
 key| _value|
+----+--------------+
123k|[BLACK, black]|
+----+--------------+

In [48]:
df.withColumn("_value2", F.expr("""filter(_value, x-> x is not null and x!='')"""))\
  .filter(~(F.size("_value2")==F.size("_value"))).drop("_value2").show()

+----+-----------------+
 key| _value|
+----+-----------------+
123k| [BLACK, black,]|
123k|[BLACK, black,, ]|
123k| [WHITE,]|
124k| [BLUE, ]|
+----+-----------------+

In [50]:

list=[[1234567,'2020-03-13 10:56:18'],
[1234567,'2020-03-12 17:09:48'],
[1234567,'2020-03-12 15:42:25'],
[1234567,'2020-03-09 16:30:22'],
[1234567,'2020-03-12 17:09:48'],
[1234567,'2020-03-09 16:30:22'],
[1234567,'2020-03-12 15:42:25']]

dfScans=spark.createDataFrame(list,['id','scandatetime'])
dfScans.show()


+-------+-------------------+
 id| scandatetime|
+-------+-------------------+
1234567|2020-03-13 10:56:18|
1234567|2020-03-12 17:09:48|
1234567|2020-03-12 15:42:25|
1234567|2020-03-09 16:30:22|
1234567|2020-03-12 17:09:48|
1234567|2020-03-09 16:30:22|
1234567|2020-03-12 15:42:25|
+-------+-------------------+

In [51]:
dfScans.groupBy("id").agg(F.min('scandatetime').alias('FirstScanDate'),
    F.max('scandatetime').alias('LastScanDate')).show()

+-------+-------------------+-------------------+
 id| FirstScanDate| LastScanDate|
+-------+-------------------+-------------------+
1234567|2020-03-09 16:30:22|2020-03-13 10:56:18|
+-------+-------------------+-------------------+

In [52]:
list=[['q1'    ,['url7', 'url11', 'url12', 'url13', 'url14']],
      ['q2'    ,['url1', 'url3', 'url5', 'url6']],
      ['q3'    ,['url1', 'url2', 'url8',None]]]
df=spark.createDataFrame(list, ['query','href'])

df.show(truncate=False)

urls = ['url1', 'url2', 'url3', 'url4', 'url5', 'url6', 'url7', 'url8',None]

+-----+----------------------------------+
query|href |
+-----+----------------------------------+
q1 |[url7, url11, url12, url13, url14]|
q2 |[url1, url3, url5, url6] |
q3 |[url1, url2, url8,] |
+-----+----------------------------------+

In [53]:
df1=df.select("query", F.explode("href").alias("col"))
df=df1.join(df, ['query']).withColumn("position", F.expr("""array_position(href,col)-1"""))\
   .select("query", F.col("col").alias("href"), "position")

In [54]:
df2.count()

Out[79]: 12

In [55]:
df.agg(*[F.count(F.when(F.isnull(c), c)).alias(c) for c in df.columns], F.count(F.lit(1)).alias("t_rows"))\
   .withColumn("t_cols", F.lit(len(df.columns))).drop(*(x for x in show()

+-----+----+--------+------+------+
query|href|position|t_rows|t_cols|
+-----+----+--------+------+------+
 0| 1| 1| 13| 3|
+-----+----+--------+------+------+

In [56]:
len(df2.columns)

Out[68]: 3

In [57]:
list=[['q1'    ,['url7', 'url11', 'url12', 'url13', 'url14']],
      ['q2'    ,['url1', 'url3', 'url5', 'url6']],
      ['q3'    ,['url1', 'url2', 'url8']]]
df=spark.createDataFrame(list, ['query','href'])

df.show(truncate=False)

+-----+----------------------------------+
query|href |
+-----+----------------------------------+
q1 |[url7, url11, url12, url13, url14]|
q2 |[url1, url3, url5, url6] |
q3 |[url1, url2, url8] |
+-----+----------------------------------+

In [58]:
urls=['url1', 'url2', 'url3', 'url4', 'url5', 'url6', 'url7', 'url8']
df.withColumn("href1", F.col("href")).select("query", F.explode("href").alias("col"),"href1")\
  .withColumn("position", F.expr("""array_position(href1,col)-1""")).drop("href1")\
  .withColumn("array", F.array(*(F.lit(x) for x in urls)))\
  .withColumn("array_contains", F.expr("""array_contains(array,col)"""))\
  .filter("array_contains= true")\
  .select("query", F.col("col").alias("href"),"position").show()

+-----+----+--------+
query|href|position|
+-----+----+--------+
 q1|url7| 0|
 q2|url1| 0|
 q2|url3| 1|
 q2|url5| 2|
 q2|url6| 3|
 q3|url1| 0|
 q3|url2| 1|
 q3|url8| 2|
+-----+----+--------+

In [59]:
.select("query", F.col("col").alias("href"),"position").show()

In [60]:
list=[[1 , 2 , 3],
  [1 , 2 , 3],
 [1 , 2 , 3]]

df=spark.createDataFrame(list,['A','B','C'])

df.show()

+---+---+---+
 A| B| C|
+---+---+---+
 1| 2| 3|
 1| 2| 3|
 1| 2| 3|
+---+---+---+

In [61]:
a=df.groupBy().agg(F.sum("B")/F.sum("A")).collect()[0][0]
a

Out[5]: 2.0

In [62]:
list=[[20200326,122450]]
df=spark.createDataFrame(list,['date','time'])
df.show()


+--------+------+
 date| time|
+--------+------+
20200326|122450|
+--------+------+

In [63]:
df.withColumn("year", F.substring("date",1,4))\
  .withColumn("month",F.substring("date",5,2))\
  .withColumn("day",F.substring("date",7,2)).drop("date")\
  .withColumn("date", F.concat_ws("/",*(x for x in ['year','month','day'])))\
  .withColumn("hour", F.substring("time", 1,2))\
  .withColumn("minute",F.substring("time", 2,2))\
  .withColumn("seconds", F.substring("time", 4,2)).drop("time")\
  .withColumn("time", F.concat_ws(":",*(x for x in ['hour','minute','seconds'])))\
  .select("date","time").show()

+----------+--------+
 date| time|
+----------+--------+
2020/03/26|12:22:45|
+----------+--------+

In [64]:



list=[['86311425-0890-40a5-8950-54cbaaa60815','2020-03-18 14:41:55 UTC'],
      ['38e121a8-f21f-4d10-bb69-26eb045175b5','2020-03-13 15:19:21 UTC'],
      ['1a69c9b0-283b-4b6d-89ac-66f987280c66','2020-03-16 12:59:51 UTC'],
      ['c7b5c53f-bf40-498f-8302-4b3329322bc9','2020-03-18 22:05:06 UTC'],
      ['0d3d807b-9b3a-466e-907c-c22402240730','2020-03-17 18:40:03 UTC']]

df=spark.createDataFrame(list,['identifier','timestamp'])

df.show(truncate=False)

+------------------------------------+-----------------------+
identifier |timestamp |
+------------------------------------+-----------------------+
86311425-0890-40a5-8950-54cbaaa60815|2020-03-18 14:41:55 UTC|
38e121a8-f21f-4d10-bb69-26eb045175b5|2020-03-13 15:19:21 UTC|
1a69c9b0-283b-4b6d-89ac-66f987280c66|2020-03-16 12:59:51 UTC|
c7b5c53f-bf40-498f-8302-4b3329322bc9|2020-03-18 22:05:06 UTC|
0d3d807b-9b3a-466e-907c-c22402240730|2020-03-17 18:40:03 UTC|
+------------------------------------+-----------------------+

In [65]:
df.withColumn("hour", F.date_trunc('hour',F.to_timestamp("timestamp","yyyy-MM-dd HH:mm:ss")))\
  .show(truncate=False)

+------------------------------------+-----------------------+-------------------+
identifier |timestamp |hour |
+------------------------------------+-----------------------+-------------------+
86311425-0890-40a5-8950-54cbaaa60815|2020-03-18 14:41:55 UTC|2020-03-18 14:00:00|
38e121a8-f21f-4d10-bb69-26eb045175b5|2020-03-13 15:19:21 UTC|2020-03-13 15:00:00|
1a69c9b0-283b-4b6d-89ac-66f987280c66|2020-03-16 12:59:51 UTC|2020-03-16 12:00:00|
c7b5c53f-bf40-498f-8302-4b3329322bc9|2020-03-18 22:05:06 UTC|2020-03-18 22:00:00|
0d3d807b-9b3a-466e-907c-c22402240730|2020-03-17 18:40:03 UTC|2020-03-17 18:00:00|
+------------------------------------+-----------------------+-------------------+

In [66]:
321869c3-71e5-41d...|2020-03-19 03:00:...|2020-03-19 03:00:00|
|226b8d50-2c6a-471...|2020-03-19 02:59:...|2020-03-19 02:00:00|
|47818b7c-34b5-43c...|2020-03-19 01:41:...|2020-03-19 01:00:00|
|f5ca5599-7252-49d...|2020-03-19 04:25:...|2020-03-19 04:00:00|
|add2ae24-aa7b-4d3...|2020-03-19 01:50:...|2020-03-19 01:00:00

In [67]:
df.withColumn("new", F.to_timestamp(F.substring("timestamp", 1, 19),"yyyy-MM-dd hh:mm:ss a")).show(truncate=False)

+------------------------------------+-----------------------+----+
identifier |timestamp |new |
+------------------------------------+-----------------------+----+
86311425-0890-40a5-8950-54cbaaa60815|2020-03-18 14:41:55 UTC|null|
38e121a8-f21f-4d10-bb69-26eb045175b5|2020-03-13 15:19:21 UTC|null|
1a69c9b0-283b-4b6d-89ac-66f987280c66|2020-03-16 12:59:51 UTC|null|
c7b5c53f-bf40-498f-8302-4b3329322bc9|2020-03-18 22:05:06 UTC|null|
0d3d807b-9b3a-466e-907c-c22402240730|2020-03-17 18:40:03 UTC|null|
+------------------------------------+-----------------------+----+

In [68]:
from pyspark.sql.functions import unix_timestamp, from_unixtime

df = spark.createDataFrame(
    [("2020-02-28T09:49Z",)], 
    ['date_str']
)

df2 = df.select(
    'date_str', 
    from_unixtime(unix_timestamp('date_str', "yyyy-MM-dd'T'hh:mm'Z'")).alias('date')
)

df2.show()

+-----------------+-------------------+
 date_str| date|
+-----------------+-------------------+
2020-02-28T09:49Z|2020-02-28 09:49:00|
+-----------------+-------------------+

In [69]:

list=[[1,0.2,23.78,'true','hello_world'],
      [2,0.6,12.41,'false','byebye_world']]
  
df= spark.createDataFrame(list, ['id','score','tx_amount','isValid','greeting'])

df.show()


+---+-----+---------+-------+------------+
 id|score|tx_amount|isValid| greeting|
+---+-----+---------+-------+------------+
 1| 0.2| 23.78| true| hello_world|
 2| 0.6| 12.41| false|byebye_world|
+---+-----+---------+-------+------------+

In [71]:
from pyspark.sql import functions as F
df.withColumn("cols", F.explode(F.arrays_zip(F.array("score", "tx_amount", "isValid", "greeting")))) \
        .select("id", F.col("cols.*")) \
        .withColumnRenamed("0", "col_value")\
        .withColumn("text", (F.regexp_extract(F.col("col_value"),"([A-Za-z]+)",1)))\
        .withColumn("boolean", F.when((F.col("text")=='true')|(F.col("text")=='false'),F.col("text")).otherwise(F.lit("")))\
        .withColumn("text", F.when(F.col("text")==F.col("boolean"), F.lit("")).otherwise(F.col("text")))\
        .withColumn("numeric", F.regexp_extract(F.col("col_value"),"(^[.?\d]+$)",1))\
        .withColumn("is_text", F.when(F.col("text")!="", F.lit("Y")).otherwise(F.lit("N")))\
        .withColumn("is_score", F.when(F.col("numeric")<=1, F.lit("Y")).otherwise(F.lit("N")))\
        .withColumn("is_amount", F.when(F.col("numeric")>1, F.lit("Y")).otherwise(F.lit("N")))\
        .withColumn("is_boolean", F.when(F.col("boolean")!="", F.lit("Y")).otherwise(F.lit("N")))\
        .select("id", "col_value","is_score","is_amount","is_boolean","is_text").show()

+---+------------+--------+---------+----------+-------+
 id| col_value|is_score|is_amount|is_boolean|is_text|
+---+------------+--------+---------+----------+-------+
 1| 0.2| Y| N| N| N|
 1| 23.78| N| Y| N| N|
 1| true| N| N| Y| N|
 1| hello_world| N| N| N| Y|
 2| 0.6| Y| N| N| N|
 2| 12.41| N| Y| N| N|
 2| false| N| N| Y| N|
 2|byebye_world| N| N| N| Y|
+---+------------+--------+---------+----------+-------+

In [72]:
from pyspark.sql import functions as F
df.withColumn("cols", F.explode(F.arrays_zip(F.array("score", "tx_amount", "isValid", "greeting")))) \
        .select("id", F.col("cols.*")) \
        .withColumnRenamed("0", "col_value")\
        .withColumn("text", (F.regexp_extract(F.col("col_value"),"([A-Za-z]+)",1)))\
        .withColumn("boolean", F.when((F.col("text")=='true')|(F.col("text")=='false'),F.col("text")).otherwise(F.lit("")))\
        .withColumn("text", F.when(F.col("text")==F.col("boolean"), F.lit("")).otherwise(F.col("text")))\
        .withColumn("numeric", F.regexp_extract(F.col("col_value"),"(^[.?\d]+$)",1)).show()

+---+------------+------+-------+-------+
 id| col_value| text|boolean|numeric|
+---+------------+------+-------+-------+
 1| 0.2| | | 0.2|
 1| 23.78| | | 23.78|
 1| true| | true| |
 1| hello_world| hello| | |
 2| 0.6| | | 0.6|
 2| 12.41| | | 12.41|
 2| false| | false| |
 2|byebye_world|byebye| | |
+---+------------+------+-------+-------+

In [73]:
+---+------------+--------+---------+----------+-------+
| id|   col_value|is_score|is_amount|is_boolean|is_text|
+---+------------+--------+---------+----------+-------+
|  1|         0.2|       Y|        N|         N|      N|
|  1|       23.78|       N|        Y|         N|      N|
|  1|        true|       N|        N|         Y|      N|
|  1| hello_world|       N|        N|         N|      Y|
|  2|         0.6|       Y|        N|         N|      N|
|  2|       12.41|       N|        Y|         N|      N|
|  2|       false|       N|        N|         Y|      N|
|  2|byebye_world|       N|        N|         N|      Y|
+---+------------+--------+---------+----------+-------+

In [74]:
df = spark.createDataFrame([
                            (1,'st' ,'aaa'), (1,'st', 'bbb'),
                            (2,'ac' ,'aaa'), (2,'ac', 'ccc'), (3,'ac', 'ggg'),
                            (3,'ds' ,'aaa'), (3,'ds', 'ggg'),(3,'ds', 'mmm')
                            ], ("id","code" ,"prod")) 
df.show()

pairId = spark.createDataFrame([('st' ,'ac'), ('st' ,'ac'),('st','ds' ),('st','ds' ),('ac' ,'ds' )], 
                               ("idSorce","idTarget" ))

pairId .show()

+---+----+----+
 id|code|prod|
+---+----+----+
 1| st| aaa|
 1| st| bbb|
 2| ac| aaa|
 2| ac| ccc|
 3| ac| ggg|
 3| ds| aaa|
 3| ds| ggg|
 3| ds| mmm|
+---+----+----+

+-------+--------+
idSorce|idTarget|
+-------+--------+
 st| ac|
 st| ac|
 st| ds|
 st| ds|
 ac| ds|
+-------+--------+

In [75]:
df.join(pairId.select("idTarget"), df.code==pairId.idTarget,'inner').show()

+---+----+----+--------+
 id|code|prod|idTarget|
+---+----+----+--------+
 3| ds| aaa| ds|
 3| ds| aaa| ds|
 3| ds| aaa| ds|
 3| ds| ggg| ds|
 3| ds| ggg| ds|
 3| ds| ggg| ds|
 3| ds| mmm| ds|
 3| ds| mmm| ds|
 3| ds| mmm| ds|
 2| ac| aaa| ac|
 2| ac| aaa| ac|
 2| ac| ccc| ac|
 2| ac| ccc| ac|
 3| ac| ggg| ac|
 3| ac| ggg| ac|
+---+----+----+--------+

In [77]:
list=[[235832,'EXTREME BERRY Sweet'],          
      [419736,'BLUE CHASER SAUCE'],             
      [124513,'LAAVA C2L5'],
      [123455,'LAVA']]         

df=spark.createDataFrame(list,['id','productname'])

from pyspark.sql import functions as F
from pyspark.sql.functions import when
df.withColumn("array", F.split("productname","\ "))\
  .withColumn("array", F.when(F.size("array")==2, F.array_union(F.col("array"),F.array(F.lit(""))))\
                        .when(F.size("array")==1, F.array_union(F.col("array"),F.array(F.lit(" "),F.lit(""))))\
                        .otherwise(F.col("array")))\
  .withColumn("product1", F.element_at("array",1))\
  .withColumn("product2", F.element_at("array",2))\
  .withColumn("product3", F.element_at("array",3)).drop("array")\
  .show(truncate=False)

+------+-------------------+--------+--------+--------+
id |productname |product1|product2|product3|
+------+-------------------+--------+--------+--------+
235832|EXTREME BERRY Sweet|EXTREME |BERRY |Sweet |
419736|BLUE CHASER SAUCE |BLUE |CHASER |SAUCE |
124513|LAAVA C2L5 |LAAVA |C2L5 | |
123455|LAVA |LAVA | | |
+------+-------------------+--------+--------+--------+

In [78]:
+------+-------------------+-------------+-------------+-------------+
|id    |productname        |product1     |product2     |product3     |
+------+-------------------+-------------+-------------+-------------+
|235832|EXTREME BERRY Sweet|EXTREME      |BERRY        |Sweet        |
|419736|BLUE CHASER SAUCE  |BLUE         |CHASER       |SAUCE        |
|124513|LAAVA C2L5         |LAAVA        |C2L5         |             |
+------+-------------------+-------------+-------------+-------------+

In [79]:
list=[['Alex'         ,["Physics","Math","biology"]],
      ['Sam'          ,["Economics","History","Math","Physics"]],
      ['Claire'      , ["Political science","Physics"]],
      ['Claire'      , ["Math","Physics"]],
      ['Claire'      , ["Math"]],
      ['Claire'      , []]]

df=spark.createDataFrame(list,['Studentname','Speciality'])

df.show(truncate=False)

+-----------+-----------------------------------+
Studentname|Speciality |
+-----------+-----------------------------------+
Alex |[Physics, Math, biology] |
Sam |[Economics, History, Math, Physics]|
Claire |[Political science, Physics] |
Claire |[Math, Physics] |
Claire |[Math] |
Claire |[] |
+-----------+-----------------------------------+

In [80]:
df.withColumn("Math",F.lit("Math")).withColumn("Physics", F.lit("Physics"))\
  .withColumn("new", F.size(F.expr("""filter(Speciality, x-> x== Math or x== Physics)""")))\
  .filter("new=2").drop("Math","Physics","new").show(truncate=False)

+-----------+-----------------------------------+
Studentname|Speciality |
+-----------+-----------------------------------+
Alex |[Physics, Math, biology] |
Sam |[Economics, History, Math, Physics]|
Mo |[Political science, Physics, Math] |
+-----------+-----------------------------------+

In [81]:
a=['Math','Physics']

df.withColumn("new", F.size(F.expr("""filter(Speciality, x-> x=='Math' or x== 'Physics')""")))\
  .filter("new=2").show(truncate=False)

+-----------+-----------------------------------+---+
Studentname|Speciality |new|
+-----------+-----------------------------------+---+
Alex |[Physics, Math, biology] |2 |
Sam |[Economics, History, Math, Physics]|2 |
+-----------+-----------------------------------+---+

In [82]:
a1=['Math','Physics']
df.withColumn("array", F.array_except("Speciality",F.array(*(F.lit(x) for x in a1))))\
  .filter("size(array) = size(Speciality)-2").show(truncate=False)

+-----------+-----------------------------------+--------------------+
Studentname|Speciality |array |
+-----------+-----------------------------------+--------------------+
Alex |[Physics, Math, biology] |[biology] |
Sam |[Economics, History, Math, Physics]|[Economics, History]|
Claire |[Math, Physics] |[] |
+-----------+-----------------------------------+--------------------+

In [84]:
+-----------+-----------------------------------+
|Studentname|Speciality                         |
+-----------+-----------------------------------+
|Alex       |[Physics, Math, biology]           |
|Sam        |[Economics, History, Math, Physics]|
+-----------+-----------------------------------+

In [85]:
list=[[234832,'EXTREME BERRY SAUCE'],
[419836,'BLUE KOSHER SAUCE'],
[350022,'GUAVA (1G)'],
[123213,'GUAVA 1G']] 


list1=[[234832,'EXTREME BERRY SAUCE'],
[419836,'BLUE KOSHER SAUCE'],
[350022,'GUAVA (1G)'], 
[123213,'GUAVA G5'],           
[125513,'3GULA G5' ],          
[127143,'GUAVA G50' ],        
[124513,'LAAVA C2L5' ]]        

df=spark.createDataFrame(list1,['id','productname'])
df.show()


+------+-------------------+
 id| productname|
+------+-------------------+
234832|EXTREME BERRY SAUCE|
419836| BLUE KOSHER SAUCE|
350022| GUAVA (1G)|
123213| GUAVA G5|
125513| 3GULA G5|
127143| GUAVA G50|
124513| LAAVA C2L5|
+------+-------------------+

In [86]:
df.withColumn("product1_num", F.regexp_extract("productname", "([0-9]+)",1)).show()

+------+-------------------+------------+
 id| productname|product1_num|
+------+-------------------+------------+
234832|EXTREME BERRY SAUCE| |
419836| BLUE KOSHER SAUCE| |
350022| GUAVA (1G)| 1|
123213| GUAVA G5| 5|
125513| 3GULA G5| 3|
127143| GUAVA G50| 50|
124513| LAAVA C2L5| 2|
+------+-------------------+------------+

In [87]:
list=[[ 10          , 1      , 'ABC'   , 'Core'],
      [200          , 1      , 'ABC'  , 'Home'],
      [300     , 9      , 'ABC'   , 'Core'],
      [15     , 9      , 'ABC'   , 'Managed'],
      [10         , 9      , 'ABC'   , 'Home'],
      [100          , 131   , 'MNM'   , 'Home'],
      [200         , 199    , 'KJK'   , 'Home']] 
df=spark.createDataFrame(list, ['property_count','prov_id','co_id','segment'])
df.show()

+--------------+-------+-----+-------+
property_count|prov_id|co_id|segment|
+--------------+-------+-----+-------+
 10| 1| ABC| Core|
 200| 1| ABC| Home|
 300| 9| ABC| Core|
 15| 9| ABC|Managed|
 10| 9| ABC| Home|
 100| 131| MNM| Home|
 200| 199| KJK| Home|
+--------------+-------+-----+-------+

In [88]:
w1=Window().partitionBy("prov_id","segment")
w2=Window().partitionBy("co_id","segment")
w3=Window().partitionBy("prov_id")
w4=Window().partitionBy("co_id")

df.withColumn("prov_sum", F.sum("property_count").over(w1))\
  .withColumn("co_sum", F.sum("property_count").over(w2))\
  .withColumn("prov_total", F.sum("property_count").over(w3))\
  .withColumn("co_total", F.sum("property_count").over(w4))\
  .withColumn("prov_percentage", (F.col("prov_sum")/F.col("prov_total")*100))\
  .withColumn("co_percentage", (F.col("co_sum")/F.col("co_total"))*100)\
  .withColumn("prov_percentage", F.when(F.col("prov_percentage")>50, F.col("prov_percentage")).otherwise(F.lit(None)))\
  .withColumn("co_percentage", F.when(F.col("co_percentage")>50, F.col("co_percentage")).otherwise(F.lit(None)))\
  .drop("property_count","prov_sum","co_sum","prov_total","co_total")\
  .withColumn("first1", F.first("prov_percentage", True).over(w3))\
  .withColumn("first2", F.first("co_percentage",True).over(w4))\
  .withColumn("prov_segment", F.when(F.col("prov_percentage")==F.col("first1"),F.col("segment")).otherwise(F.lit(None)))\
  .withColumn("co_segment", F.when(F.col("co_percentage")==F.col("first2"),F.col("segment")).otherwise(F.lit(None)))\
  .select("prov_id","prov_segment","co_id","co_segment")\
  .groupBy("prov_id","co_id").agg(*(F.first(x, True).alias(x) for x in ['prov_segment','co_segment']))\
  .select("prov_id","prov_segment","co_id","co_segment").orderBy("prov_id").show()

+-------+------------+-----+----------+
prov_id|prov_segment|co_id|co_segment|
+-------+------------+-----+----------+
 1| Home| ABC| Core|
 9| Core| ABC| Core|
 131| Home| MNM| Home|
 199| Home| KJK| Home|
+-------+------------+-----+----------+

In [89]:
w2=Window().partitionBy("co_id","segment")
w3=Window().partitionBy("prov_id")
w4=Window().partitionBy("co_id")

df.withColumn("prov_sum", F.sum("property_count").over(w1))\
  .withColumn("co_sum", F.sum("property_count").over(w2))\
  .withColumn("prov_total", F.sum("property_count").over(w3))\
  .withColumn("co_total", F.sum("property_count").over(w4))\
  .withColumn("prov_percentage", (F.col("prov_sum")/F.col("prov_total")*100))\
  .withColumn("co_percentage", (F.col("co_sum")/F.col("co_total"))*100).show()

+--------------+-------+-----+-------+--------+------+----------+--------+------------------+------------------+
property_count|prov_id|co_id|segment|prov_sum|co_sum|prov_total|co_total| prov_percentage| co_percentage|
+--------------+-------+-----+-------+--------+------+----------+--------+------------------+------------------+
 200| 199| KJK| Home| 200| 200| 200| 200| 100.0| 100.0|
 100| 131| MNM| Home| 100| 100| 100| 100| 100.0| 100.0|
 10| 9| ABC| Home| 10| 210| 310| 1420| 3.225806451612903|14.788732394366196|
 300| 9| ABC| Core| 300| 310| 310| 1420| 96.7741935483871|21.830985915492956|
 200| 1| ABC| Home| 200| 210| 1110| 1420| 18.01801801801802|14.788732394366196|
 10| 1| ABC| Core| 10| 310| 1110| 1420|0.9009009009009009|21.830985915492956|
 900| 1| ABC|Managed| 900| 900| 1110| 1420| 81.08108108108108| 63.38028169014085|
+--------------+-------+-----+-------+--------+------+----------+--------+------------------+------------------+

In [90]:
df2= df1.groupBy("prov_id")

In [91]:
| prov_id | prov_segment | co_id | co_segment |
|---------|--------------|-------|------------|
| 1       | Home         | ABC   | Core       |
| 9       | Core         | ABC   | Core       |
| 131     | Home         | MNM   | Home       |
| 199     | Home         | KJK   | Home       |

In [92]:
w1=Window().partitionBy("prov_id","segment")
w2=Window().partitionBy("co_id","segment")
w3=Window().partitionBy("prov_id")
w4=Window().partitionBy("co_id")

df1=df.withColumn("prov_sum", F.sum("property_count").over(w1))\
  .withColumn("co_sum", F.sum("property_count").over(w2))\
  .withColumn("prov_total", F.sum("property_count").over(w3))\
  .withColumn("co_total", F.sum("property_count").over(w4))\
  .withColumn("prov_percentage", (F.col("prov_sum")/F.col("prov_total")*100))\
  .withColumn("co_percentage", (F.col("co_sum")/F.col("co_total"))*100)\
  .withColumn("prov_percentage", F.when(F.col("prov_percentage")>50, F.col("prov_percentage")).otherwise(F.lit(None)))\
  .withColumn("co_percentage", F.when(F.col("co_percentage")>50, F.col("co_percentage")).otherwise(F.lit(None)))\
  .select("prov_id","co_id","segment","prov_percentage", "co_percentage")

+-------+-----+-------+-----------------+-----------------+
prov_id|co_id|segment| prov_percentage| co_percentage|
+-------+-----+-------+-----------------+-----------------+
 199| KJK| Home| 100.0| 100.0|
 131| MNM| Home| 100.0| 100.0|
 9| ABC| Home| null| null|
 9| ABC| Core| 96.7741935483871|59.61538461538461|
 1| ABC| Home|95.23809523809523| null|
 1| ABC| Core| null|59.61538461538461|
+-------+-----+-------+-----------------+-----------------+

In [93]:
df1.groupBy("prov_id")

In [94]:
list=[[0 ,  1  , '01/22/2020'],
     [2  , 3  , '01/22/2020'],  
     [3  , 4  , '01/22/2020'],  
     [4  , 5  , '03/22/2020']]  


df=spark.createDataFrame(list, ['1','2','date'])

df.withColumn("date", F.to_date("date", "MM/dd/yyyy")).select(F.max("date")).collect()[0][0]

Out[5]: datetime.date(2020, 3, 22)

In [95]:
max_date=df.select(F.max("date")).collect()[0][0]
max_date

Out[10]: '03/22/2020'

In [96]:
max_date=df.select(F.max("date")).collect()[0][0]
df.filter(F.col("date")==max_date)\
.groupBy("ObservationDate")\
.agg({'Confirmed':'sum', 'Deaths':'sum', 'Recovered':'sum', 'Active_case':'sum'})\
.show()

+---+---+----------+
 1| 2| date|
+---+---+----------+
 4| 5|03/22/2020|
+---+---+----------+

In [97]:



list=[[1,5.64822705794013, 315.320000],
     [1,  2.80033330216659, 315.320000],
     [1,   2.8222365762732, 315.320000],
     [1,  2.18430371791803025, 325.320000],
     [1, 5.686557787521759, 325.320000],
     [2,  5.70572315231258, 315.320000],
     [2, 5.724888517103389, 315.320000],
     [3, 5.744053881894209, 315.320000],
     [3,5.7604813374292005, 315.320000],
     [3,  5.77993740687426, 315.320000]]

df=spark.createDataFrame(list, ['id','diagnosis_age','diagnosis'])

df.show()


+---+------------------+---------+
 id| diagnosis_age|diagnosis|
+---+------------------+---------+
 1| 5.64822705794013| 315.32|
 1| 2.80033330216659| 315.32|
 1| 2.8222365762732| 315.32|
 1|2.1843037179180302| 325.32|
 1| 5.686557787521759| 325.32|
 2| 5.70572315231258| 315.32|
 2| 5.724888517103389| 315.32|
 3| 5.744053881894209| 315.32|
 3|5.7604813374292005| 315.32|
 3| 5.77993740687426| 315.32|
+---+------------------+---------+

In [98]:
w=Window().partitionBy("id","diagnosis").orderBy("diagnosis_age")
w2=Window().partitionBy("id")
df.withColumn("count", F.count("diagnosis").over(w))\
  .withColumn("max", F.max("count").over(w2))\
  .filter("count=max")\
  .groupBy("id").agg(F.first("diagnosis_age").alias("diagnosis_age"),F.first("diagnosis").alias("diagnosis"))\
  .orderBy("id").show()

+---+-----------------+---------+
 id| diagnosis_age|diagnosis|
+---+-----------------+---------+
 1| 5.64822705794013| 315.32|
 2|5.724888517103389| 315.32|
 3| 5.77993740687426| 315.32|
+---+-----------------+---------+

In [99]:
+---+-----------------+---------+
| id|    diagnosis_age|diagnosis|
+---+-----------------+---------+
|  1| 2.80033330216659|   315.32|
|  3|5.744053881894209|   315.32|
|  2| 5.70572315231258|   315.32|
+---+-----------------+---------+

In [100]:
w=Window().partitionBy("id").orderBy("diagnosis_age")
df.withColumn("least_age", F.first("diagnosis_age").over(w)).filter("diagnosis_age=least_age").drop("least_age").show()

+---+------------------+---------+
 id| diagnosis_age|diagnosis|
+---+------------------+---------+
 1|2.1843037179180302| 325.32|
 3| 5.744053881894209| 315.32|
 2| 5.70572315231258| 315.32|
+---+------------------+---------+

In [101]:
df.orderBy("diagnosis_age").groupBy("id").agg(F.min("diagnosis_age").alias("diagnosis_age"), F.first("diagnosis").alias("diagnosis")).show()

+---+------------------+---------+
 id| diagnosis_age|diagnosis|
+---+------------------+---------+
 1|2.1843037179180302| 325.32|
 3| 5.744053881894209| 315.32|
 2| 5.70572315231258| 315.32|
+---+------------------+---------+